In [32]:
import pandas as pd
import glob
from datetime import datetime, timedelta

In [45]:
files = glob.glob("VX_*.csv")

frames = []

for f in files:
    expiry_str = f.split("VX_")[1].split(".csv")[0] 
    expiry = pd.to_datetime(expiry_str)

    df = pd.read_csv(f)
    df.columns = df.columns.str.strip()
    df["Trade Date"] = pd.to_datetime(df["Trade Date"])
    df["expiry"] = expiry
    frames.append(df)

full = pd.concat(frames, ignore_index=True)
full = full[full["Trade Date"] <= full["expiry"]]
prices = full[["Trade Date", "expiry", "Settle"]].copy()

expiry_list = sorted(prices["expiry"].unique())

dates = sorted(prices["Trade Date"].unique())
df_dates = pd.DataFrame({"Date": dates})

def find_front_and_second(d):
    exps = [e for e in expiry_list if e >= d]
    if len(exps) == 0:
        return pd.NaT, pd.NaT
    elif len(exps) == 1:
        return exps[0], pd.NaT
    else:
        return exps[0], exps[1]

df_dates["Expiry1"], df_dates["Expiry2"] = zip(*df_dates["Date"].apply(find_front_and_second))

MONTH_CODES = {1:"F",2:"G",3:"H",4:"J",5:"K",6:"M",7:"N",8:"Q",9:"U",10:"V",11:"X",12:"Z"}

def expiry_to_ticker(exp):
    if pd.isna(exp):
        return None
    m = MONTH_CODES[exp.month]
    y = (exp.year - 2010) % 10
    return f"VX{m}{y} Comdty"

df_dates["VX1_Ticker"] = df_dates["Expiry1"].apply(expiry_to_ticker)
df_dates["VX2_Ticker"] = df_dates["Expiry2"].apply(expiry_to_ticker)

merged = df_dates.merge(
    prices.rename(columns={"Trade Date": "Date", "expiry": "Expiry1", "Settle": "VX1"}),
    on=["Date", "Expiry1"],
    how="left"
)

merged = merged.merge(
    prices.rename(columns={"Trade Date": "Date", "expiry": "Expiry2", "Settle": "VX2"}),
    on=["Date", "Expiry2"],
    how="left"
)

merged["Slope"] = (merged["VX2"] - merged["VX1"]) / merged["VX1"]

merged = merged[merged["Date"] >= pd.Timestamp("2016-01-01")]

result = merged[["Date", "VX1_Ticker", "VX2_Ticker", "VX1", "VX2", "Slope"]]
print(result.head())


          Date   VX1_Ticker   VX2_Ticker     VX1     VX2     Slope
179 2016-01-04  VXF6 Comdty  VXG6 Comdty  19.825  19.525 -0.015132
180 2016-01-05  VXF6 Comdty  VXG6 Comdty  19.425  19.275 -0.007722
181 2016-01-06  VXF6 Comdty  VXG6 Comdty  20.325  19.825 -0.024600
182 2016-01-07  VXF6 Comdty  VXG6 Comdty  23.525  21.775 -0.074389
183 2016-01-08  VXF6 Comdty  VXG6 Comdty  25.275  23.075 -0.087043


In [46]:
result.to_csv("VIX_UX1_UX2_slope_daily.csv", index=False)